In [1]:
import torch
import pyro

pyro.set_rng_seed(101)

In [2]:
loc = 0.
scale = 1.
normal = torch.distributions.Normal(loc, scale)

In [3]:
normal.sample_n(100,)

/anaconda3/envs/Computational_methods/lib/python3.6/site-packages/torch/distributions/distribution.py:133: UserWarning: sample_n will be deprecated. Use .sample((n,)) instead
  warnings.warn('sample_n will be deprecated. Use .sample((n,)) instead', UserWarning)


tensor([-0.6028, -0.1208, -0.4323,  1.8811, -0.7158,  0.0026,  0.2056,  0.5626,
         0.2795, -1.0873,  0.0613,  0.9088, -0.4134, -0.1937, -2.1826,  0.7185,
        -1.4568, -0.9884,  1.0552,  0.7783,  1.0674,  0.1143, -0.5580,  0.3066,
        -0.6245, -0.1707,  0.9192,  0.5596,  1.7718, -0.9621,  0.0615, -0.9060,
        -0.9234,  0.3869, -1.0650, -0.2427, -1.1875,  0.8563, -0.4388, -0.5191,
         0.3763, -0.8023, -0.5348, -0.8881,  0.7324, -0.9758,  1.5274, -0.5501,
         1.5084,  1.0319, -0.0614, -0.0825,  0.0899, -0.9126,  0.2860,  0.1643,
         1.3077, -0.2920, -1.2541,  0.7954, -0.8233, -0.2827, -0.5396,  0.3486,
        -0.5913,  0.9778, -1.0367,  0.0707,  0.2845,  0.8097,  1.5251,  0.7898,
         0.1300, -0.6862, -0.3447, -0.0321, -2.3304, -0.6717,  2.1601, -0.5520,
         1.1598,  0.0898, -0.8553,  1.6752, -0.0120, -0.1540,  0.5874, -0.6361,
         0.4733, -0.9559, -0.4316, -0.5122,  0.7646, -1.0890, -1.0525, -1.2369,
         0.0899, -1.4896,  0.7798, -0.76

In [4]:
loc = 0.   # mean zero
scale = 1. # unit variance
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
x = normal.rsample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(0.2813)
log prob tensor(-0.9585)


In [5]:
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny':75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

In [6]:
weather()

('cloudy', 58.95573043823242)

In [7]:
for _ in range(20):
    print(weather())

('cloudy', 53.66604232788086)
('sunny', 81.31587219238281)
('sunny', 68.67396545410156)
('cloudy', 59.35697937011719)
('sunny', 81.7517318725586)
('cloudy', 41.12607192993164)
('sunny', 52.129005432128906)
('sunny', 72.03153991699219)
('cloudy', 39.4169921875)
('sunny', 72.46565246582031)
('sunny', 52.986541748046875)
('sunny', 71.45484924316406)
('cloudy', 70.47327423095703)
('sunny', 77.05457305908203)
('sunny', 55.338890075683594)
('sunny', 72.42037963867188)
('cloudy', 59.5177001953125)
('cloudy', 46.072330474853516)
('sunny', 67.05213165283203)
('sunny', 64.07766723632812)


In [8]:
x = pyro.sample("my_sample", pyro.distributions.Normal(loc,scale))
print(x)

tensor(-2.0339)


In [9]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.5))

0


In [10]:
out = []
for _ in range(100):
    out.append(geometric(.5))

import matplotlib.pyplot as plt

plt.hist(out)

(array([51., 26.,  7.,  0.,  7.,  5.,  0.,  2.,  1.,  1.]),
 array([0. , 0.7, 1.4, 2.1, 2.8, 3.5, 4.2, 4.9, 5.6, 6.3, 7. ]),
 <a list of 10 Patch objects>)

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

pyro.set_rng_seed(101)

In [13]:

from __future__ import print_function
import os
from collections import defaultdict
import numpy as np
import scipy.stats
import torch
from torch.distributions import constraints
from matplotlib import pyplot
%matplotlib inline

import pyro
import pyro.distributions as dist
from pyro import poutine
from pyro.contrib.autoguide import AutoDelta
from pyro.optim import Adam
from pyro.infer import SVI, TraceEnum_ELBO, config_enumerate, infer_discrete

smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('0.3.0')
pyro.enable_validation(True)

ModuleNotFoundError: No module named 'scipy'

In [12]:
K = 2  # Fixed number of components.

@config_enumerate
def model(data):
    # Global variables.
    weights = pyro.sample('weights', dist.Dirichlet(0.5 * torch.ones(K)))
    scale = pyro.sample('scale', dist.LogNormal(0., 2.))
    with pyro.plate('components', K):
        locs = pyro.sample('locs', dist.Normal(0., 10.))

    with pyro.plate('data', len(data)):
        # Local variables.
        assignment = pyro.sample('assignment', dist.Categorical(weights))
        pyro.sample('obs', dist.Normal(locs[assignment], scale), obs=data)

global_guide = AutoDelta(poutine.block(model, expose=['weights', 'locs', 'scale']))

NameError: name 'config_enumerate' is not defined